In [1]:
#Sagemaker Endpoint Deploy
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()
# Hub Model configuration. https://huggingface.co/models
hub = {
	#'HF_MODEL_ID':'shibing624/text2vec-base-chinese',
    'HF_MODEL_ID':'shibing624/text2vec-large-chinese',
	'HF_TASK':'feature-extraction'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.17.0',
	pytorch_version='1.10.2',
	py_version='py38',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	endpoint_name='huggingface-inference-text2vec-base-chinese-v1',
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

----!

In [2]:
#Preprocess Data
import pandas as pd
import sagemaker
from sagemaker.huggingface import HuggingFaceModel
import json
import boto3
import requests
import json
import numpy as np

def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    similarity_score = dot_product / (norm_vector1 * norm_vector2)
    return similarity_score

def get_vector(q):
    return hfp.predict({'inputs':[q]})[0][0][0]

hfp = sagemaker.huggingface.model.HuggingFacePredictor('huggingface-inference-text2vec-base-chinese-v1')



path = 'material.csv'
df = pd.read_excel(path)
df['question_vector'] = ''
data = df['question'].tolist()
for i in range(len(df)):
    df.iloc[i, 2] = str(get_vector(df.iloc[i, 0]))
    print('\r%i out of %i finished'%(i, len(df)), end='')

test = "我想去米亚罗"
test_embedding = get_vector(test)
q_array = ["我很喜欢米亚罗的风景","我去过米亚罗","我有意向去米亚罗旅游","米亚罗下雪了"]
for q in q_array:
    q_embeding=get_vector(q)
    score = cosine_similarity(test_embedding,q_embeding)
    print(f"{}, {} ,simlarity:{}".format(test_embedding,q_embeding,score))

2672 out of 2673 finished